In [1]:
import json
import numpy
from collections import defaultdict
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time

In [2]:
import os
import baseline
import pandas as pd
import numpy as np
#TEMPORARILY SUPPRESS WARNINGS
import warnings

warnings.filterwarnings("ignore")

# get the actual labels in the validation set

In [3]:
captions_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_train2017.json"
captions_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_val2017.json"
#captions_train  = json.loads(open(captions_train_path).read())

In [4]:
inst_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_train2017.json"
inst_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_val2017.json"
#inst_train  = json.loads(open(inst_train_path).read())

In [5]:
def get_electronics_info(captions_path, inst_path):
    captions  = json.loads(open(captions_path).read())
    inst  = json.loads(open(inst_path).read())
    image_df = pd.DataFrame(inst['images'])
    annotation_df = pd.DataFrame(inst['annotations'])
    #the 'id' in image_df needs to be changed to 'image_id' in order to join with annotations_df
    renamed_image_df = image_df.copy(deep=True)
    renamed_image_df.rename(columns={'id':'image_id'}, inplace=True)
    images_and_annotations_df = annotation_df.merge(renamed_image_df,on='image_id', how='left')
    electronics_only_merged_df = images_and_annotations_df.loc[(images_and_annotations_df['category_id'] >= 72) & (images_and_annotations_df['category_id'] <= 77)]
    
    #select all images that contain electronics
    #all_images_with_electronics = list(pd.Series(electronics_only_merged_df['image_id']).unique())
    
    return(electronics_only_merged_df)

In [6]:
train_electronics = get_electronics_info(captions_train_path, inst_train_path)
val_electronics = get_electronics_info(captions_val_path, inst_val_path)

In [7]:
#train_electronics.head()

In [8]:
#val_electronics.head()

In [55]:
val_electronics[val_electronics['image_id']==139]

,area,bbox,category_id,id,image_id,iscrowd,segmentation,coco_url,date_captured,file_name,flickr_url,height,license,width
24452,13244.65770,"[7.03, 167.76, 149.32, 94.87]",72,34646,139,0,"[[9.66, 167.76, 156.35, 173.04, 153.71, 256.48...",http://images.cocodataset.org/val2017/00000000...,2013-11-21 01:34:01,000000000139.jpg,http://farm9.staticflickr.com/8035/8024364858_...,426,2,640
24458,5833.11795,"[557.21, 209.19, 81.35, 78.73]",72,35802,139,0,"[[563.33, 209.19, 637.69, 209.19, 638.56, 287....",http://images.cocodataset.org/val2017/00000000...,2013-11-21 01:34:01,000000000139.jpg,http://farm9.staticflickr.com/8035/8024364858_...,426,2,640


In [13]:
#extract electronics IDs in each image
eIDs_in_images = val_electronics.groupby('image_id')['category_id'].apply(list).to_dict()
#select all images that contain electronics
train_images_with_electronics = list(pd.Series(train_electronics['image_id']).unique())
val_images_with_electronics = list(pd.Series(val_electronics['image_id']).unique())

In [14]:
#val_electronics

In [15]:
#val_images_with_electronics[:10]

In [16]:
len(val_images_with_electronics)

597

# get the validation images with electronics

In [2]:
val_folder_name = '/mnt/muthderd/MIDS/W210/data/val2017/'
all_val_filenames = os.listdir(val_folder_name)

In [18]:
max_file_id_length = len(all_val_filenames[0]) - 4#subtract 4 for the '.jpg' suffix

In [19]:
files_to_open = []
for image_id in val_images_with_electronics:
    file_id = str(image_id)
    zeros_to_add = max_file_id_length-len(file_id)
    filename = ('0'*zeros_to_add) + file_id + '.jpg'
    files_to_open.append(filename)

In [20]:
#files_to_open[:10]

In [3]:
all_val_filenames

['000000384670.jpg',
 '000000097988.jpg',
 '000000347693.jpg',
 '000000001490.jpg',
 '000000421455.jpg',
 '000000558114.jpg',
 '000000517056.jpg',
 '000000050828.jpg',
 '000000277584.jpg',
 '000000386879.jpg',
 '000000521717.jpg',
 '000000439773.jpg',
 '000000124442.jpg',
 '000000152870.jpg',
 '000000500716.jpg',
 '000000114770.jpg',
 '000000430286.jpg',
 '000000391144.jpg',
 '000000283070.jpg',
 '000000268000.jpg',
 '000000363188.jpg',
 '000000058111.jpg',
 '000000384850.jpg',
 '000000491470.jpg',
 '000000550714.jpg',
 '000000283318.jpg',
 '000000427055.jpg',
 '000000268375.jpg',
 '000000290592.jpg',
 '000000147223.jpg',
 '000000004395.jpg',
 '000000574810.jpg',
 '000000128112.jpg',
 '000000172877.jpg',
 '000000500257.jpg',
 '000000052007.jpg',
 '000000329456.jpg',
 '000000249219.jpg',
 '000000227187.jpg',
 '000000306437.jpg',
 '000000434230.jpg',
 '000000166768.jpg',
 '000000040471.jpg',
 '000000489611.jpg',
 '000000424521.jpg',
 '000000410456.jpg',
 '000000286422.jpg',
 '00000035397

# test models

In [42]:
%%capture

#coco_metric.reset()

#results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_2/'
results_folder = '/mnt/muthderd/MIDS/W210/data/val_results/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

#baseline_model = baseline.BaselineClassifier(model='faster_rcnn_resnet101_v1d_coco')
baseline_model = baseline.BaselineClassifier(model='yolo3_darknet53_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
    #image_id = int(image[:-4])
    #labels = val_electronics[val_electronics['image_id']==image_id]
    #gt_bboxes = np.array([[coord for coord in entry] for entry in labels['bbox']]).reshape(1,len(labels),4)
    ##gt_bboxes = np.array(labels['bbox'].apply(np.array))
    #gt_labels = np.array([entry for entry in labels['category_id']]).reshape(1,len(labels))
    ##gt_labels = np.array(labels['category_id'].apply(np.array))
    
    #map_object.update(bounding_boxes, class_ids.reshape(1,100), scores.reshape(1,100), gt_bboxes, gt_labels)
    coco_metric.update(bounding_boxes, class_ids, scores)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

ValueError: zero-dimensional arrays cannot be concatenated

In [153]:
#map_object.get()
coco_metric.get()

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.52s).
Accumulating evaluation results...
DONE (t=0.92s).


(['~~~~ Summary metrics ~~~~\n',
  'person',
  'bicycle',
  'car',
  'motorcycle',
  'airplane',
  'bus',
  'train',
  'truck',
  'boat',
  'traffic light',
  'fire hydrant',
  'stop sign',
  'parking meter',
  'bench',
  'bird',
  'cat',
  'dog',
  'horse',
  'sheep',
  'cow',
  'elephant',
  'bear',
  'zebra',
  'giraffe',
  'backpack',
  'umbrella',
  'handbag',
  'tie',
  'suitcase',
  'frisbee',
  'skis',
  'snowboard',
  'sports ball',
  'kite',
  'baseball bat',
  'baseball glove',
  'skateboard',
  'surfboard',
  'tennis racket',
  'bottle',
  'wine glass',
  'cup',
  'fork',
  'knife',
  'spoon',
  'bowl',
  'banana',
  'apple',
  'sandwich',
  'orange',
  'broccoli',
  'carrot',
  'hot dog',
  'pizza',
  'donut',
  'cake',
  'chair',
  'couch',
  'potted plant',
  'bed',
  'dining table',
  'toilet',
  'tv',
  'laptop',
  'mouse',
  'remote',
  'keyboard',
  'cell phone',
  'microwave',
  'oven',
  'toaster',
  'sink',
  'refrigerator',
  'book',
  'clock',
  'vase',
  'sciss

In [138]:
pred_duration

884.2778913974762

In [100]:
all_test_annotations.head()

,image_id,class_id,scores,1,2,3,4
0,000000464476,0.0,0.927936,70.859024,81.128845,162.252640,329.081268
1,000000464476,63.0,0.692063,376.342072,282.334595,660.263550,518.431458
2,000000464476,62.0,0.356848,376.498962,289.023132,657.539246,509.743530
3,000000464476,7.0,0.326432,194.305130,31.842560,372.927612,205.119370
4,000000464476,6.0,0.145672,194.305130,31.842560,372.927612,205.119370


In [139]:
#all_test_annotations.to_csv('val_result_annotations_2.csv')
all_test_annotations.to_csv('val_result_annotations.csv')

In [140]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [141]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [142]:
total_pred_counts = []#[0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies_by_class = []
num_labels_detected = []
num_labels = []
unique_labels_by_image = []
unique_preds_by_image = []
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    current_unique_preds = [0, 0, 0, 0, 0, 0]
    current_unique_labels = [0, 0, 0, 0, 0, 0]
    pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
    label_counts = [0, 0, 0, 0, 0, 0]
    
    labels = eIDs_in_images[image_id]
    for label in labels:
        label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    unique_label_count = len(unique_labels)
    unique_labels_detected = 0
    for label in unique_labels:
        #label_counts[label-72] += 1
        current_unique_labels[label-72] += 1
        if label in unique_preds:
            unique_labels_detected += 1
    for pred in unique_preds:
        #pred_counts[pred-72] += 1
        current_unique_preds[pred-72] += 1
    
    #new method: count instances of each id to take FP into account
    
    #new method: tally accuracy for each image separately
    
    #store counts of each label and pred for each image
    total_pred_counts.append(pred_counts)
    total_label_counts.append(label_counts)
    
    #store unique labels and preds
    unique_labels_by_image.append(current_unique_labels)
    unique_preds_by_image.append(current_unique_preds)
    
    #counts of unique labels and preds
    num_labels.append(unique_label_count)
    num_labels_detected.append(unique_labels_detected)
    
    #detection_accuracies.append(num_labels_detected/num_labels)
    #detection_accuracies_by_class.append(list(np.divide(current_pred_counts, current_label_counts)))

In [143]:
pred_counts = pd.DataFrame(total_pred_counts)
label_counts = pd.DataFrame(total_label_counts)
pred_counts[6] = pred_counts.sum(axis=1)
label_counts[6] = label_counts.sum(axis=1)

In [42]:
label_counts.head()

,0,1,2,3,4,5,6
0,1,0,0,0,0,0,1
1,1,0,0,0,0,0,1
2,1,0,0,0,0,0,1
3,1,0,0,0,0,0,1
4,2,0,0,0,1,0,3


In [144]:
errors = label_counts.subtract(pred_counts).abs()
error_rate_by_class = errors.sum(axis=0).divide(label_counts.sum(axis=0))
error_rate_by_class

0    0.431579
1    0.366071
2    0.314286
3    0.709091
4    0.543624
5    0.630522
6    0.476301
dtype: float64

0    0.52
1    0.50
2    0.00
3    0.00
4    0.25
5    0.50
6    0.40
dtype: float64

In [47]:
#np.divide(num_labels_detected, num_labels)

array([1. , 1. , 0. , 1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. ,
       1. , 0. , 0. , 0. , 0. , 1. , 1. , 1. , 0. , 1. , 1. ])

In [145]:
#average percentage of unique electronics labels identified in a picture
np.mean(np.divide(num_labels_detected, num_labels))

0.5712962962962963

In [146]:
num_images = len(num_labels)
num_images

576

In [147]:
#which of the unique labels were predicted?
unique_labels_detected = (pd.DataFrame(unique_labels_by_image).astype(bool) & pd.DataFrame(unique_preds_by_image).astype(bool)).multiply(1).astype(int)
unique_labels_detected.sum(axis=0)

0    141
1    124
2     65
3     68
4     66
5     88
dtype: int64

In [148]:
num_images_labeled = pd.DataFrame(unique_labels_by_image).sum(axis=0)
num_images_labeled

0    204
1    176
2     87
3    138
4    103
5    205
dtype: int64

In [149]:
#overall rate of detecting that a label is present
unique_labels_detected.sum(axis=0).sum()/num_images_labeled.sum()

0.6046002190580504

In [150]:
#for each class, what fraction of images containing that class are predicted to contain that class?
unique_labels_detected.sum(axis=0).divide(num_images_labeled)

0    0.691176
1    0.704545
2    0.747126
3    0.492754
4    0.640777
5    0.429268
dtype: float64

In [151]:
#average rate of detecting presence of a label, by class
unique_labels_detected.sum(axis=0).divide(num_images_labeled).mean()

0.6176078294692929

In [75]:
num_images_detected = pd.DataFrame(unique_preds_by_image).sum(axis=0)
num_images_detected

0    11
1     3
2     2
3     1
4     3
5     4
dtype: int64

In [62]:
np.divide(num_images_detected, num_images_labeled)

0    0.647059
1    1.500000
2    1.000000
3    1.000000
4    1.000000
5    0.666667
dtype: float64

In [63]:
num_images_labeled.sum()

31

In [31]:
%%capture

results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_3/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

baseline_model = baseline.BaselineClassifier(model='ssd_512_resnet50_v1_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

In [32]:
pred_duration

564.4249608516693

In [33]:
all_test_annotations[:10]

,image_id,class_id,scores,1,2,3,4
0,000000464476,62.0,0.994084,377.621948,288.606720,652.204346,508.373383
1,000000464476,0.0,0.991716,69.142776,77.343102,159.968185,331.558929
2,000000464476,62.0,0.315119,367.770813,51.634697,624.718262,280.566498
3,000000464476,6.0,0.197120,195.155273,32.164742,373.132996,200.863678
4,000000464476,0.0,0.105416,527.026184,351.558197,537.178528,362.751556
5,000000464476,0.0,0.098738,520.610962,352.366943,528.946655,362.981628
6,000000464476,0.0,0.096872,530.714355,351.537720,541.468994,362.626953
7,000000464476,0.0,0.090418,548.727539,353.481445,557.438843,363.910278
8,000000464476,62.0,0.084859,195.168060,31.367561,373.370697,204.342865
9,000000464476,62.0,0.081976,363.247223,109.605972,589.127808,501.856873


In [34]:
all_test_annotations.to_csv('val_result_annotations_3.csv')

In [35]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [36]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [37]:
total_pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = [0, 0, 0, 0, 0, 0]
pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
label_counts = [0, 0, 0, 0, 0, 0]
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    labels = eIDs_in_images[image_id]
    for label in labels:
        total_label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        total_pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    for label in unique_labels:
        label_counts[label-72] += 1
    for pred in unique_preds:
        pred_counts[pred-72] += 1

In [38]:
total_pred_counts

[212, 190, 76, 90, 98, 113]

In [39]:
total_label_counts

[288, 230, 106, 282, 153, 259]

In [40]:
np.divide(total_pred_counts, total_label_counts)

array([0.73611111, 0.82608696, 0.71698113, 0.31914894, 0.64052288,
       0.43629344])

In [41]:
pred_counts

[172, 156, 62, 62, 78, 96]

In [42]:
label_counts

[207, 182, 88, 144, 106, 211]

In [43]:
np.divide(pred_counts, label_counts)

array([0.83091787, 0.85714286, 0.70454545, 0.43055556, 0.73584906,
       0.4549763 ])

In [44]:
sum(pred_counts)/sum(label_counts)

0.6673773987206824

In [ ]:
#"""Train YOLOv3 with random shapes."""
import argparse
import os
import logging
import time
import warnings
import numpy as np
import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet import autograd
import gluoncv as gcv
from gluoncv import data as gdata
from gluoncv import utils as gutils
from gluoncv.model_zoo import get_model
from gluoncv.data.batchify import Tuple, Stack, Pad
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultTrainTransform
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultValTransform
from gluoncv.data.dataloader import RandomTransformDataLoader
from gluoncv.utils.metrics.voc_detection import VOC07MApMetric
from gluoncv.utils.metrics.coco_detection import COCODetectionMetric
from gluoncv.utils import LRScheduler, LRSequential

def parse_args():
    parser = argparse.ArgumentParser(description='Train YOLO networks with random input shape.')
    parser.add_argument('--network', type=str, default='darknet53',
                        help="Base network name which serves as feature extraction base.")
    parser.add_argument('--data-shape', type=int, default=416,
                        help="Input data shape for evaluation, use 320, 416, 608... " +
                             "Training is with random shapes from (320 to 608).")
    parser.add_argument('--batch-size', type=int, default=64,
                        help='Training mini-batch size')
    parser.add_argument('--dataset', type=str, default='voc',
                        help='Training dataset. Now support voc.')
    parser.add_argument('--num-workers', '-j', dest='num_workers', type=int,
                        default=4, help='Number of data workers, you can use larger '
                        'number to accelerate data loading, if you CPU and GPUs are powerful.')
    parser.add_argument('--gpus', type=str, default='0',
                        help='Training with GPUs, you can specify 1,3 for example.')
    parser.add_argument('--epochs', type=int, default=200,
                        help='Training epochs.')
    parser.add_argument('--resume', type=str, default='',
                        help='Resume from previously saved parameters if not None. '
                        'For example, you can resume from ./yolo3_xxx_0123.params')
    parser.add_argument('--start-epoch', type=int, default=0,
                        help='Starting epoch for resuming, default is 0 for new training.'
                        'You can specify it to 100 for example to start from 100 epoch.')
    parser.add_argument('--lr', type=float, default=0.001,
                        help='Learning rate, default is 0.001')
    parser.add_argument('--lr-mode', type=str, default='step',
                        help='learning rate scheduler mode. options are step, poly and cosine.')
    parser.add_argument('--lr-decay', type=float, default=0.1,
                        help='decay rate of learning rate. default is 0.1.')
    parser.add_argument('--lr-decay-period', type=int, default=0,
                        help='interval for periodic learning rate decays. default is 0 to disable.')
    parser.add_argument('--lr-decay-epoch', type=str, default='160,180',
                        help='epochs at which learning rate decays. default is 160,180.')
    parser.add_argument('--warmup-lr', type=float, default=0.0,
                        help='starting warmup learning rate. default is 0.0.')
    parser.add_argument('--warmup-epochs', type=int, default=0,
                        help='number of warmup epochs.')
    parser.add_argument('--momentum', type=float, default=0.9,
                        help='SGD momentum, default is 0.9')
    parser.add_argument('--wd', type=float, default=0.0005,
                        help='Weight decay, default is 5e-4')
    parser.add_argument('--log-interval', type=int, default=100,
                        help='Logging mini-batch interval. Default is 100.')
    parser.add_argument('--save-prefix', type=str, default='',
                        help='Saving parameter prefix')
    parser.add_argument('--save-interval', type=int, default=10,
                        help='Saving parameters epoch interval, best model will always be saved.')
    parser.add_argument('--val-interval', type=int, default=1,
                        help='Epoch interval for validation, increase the number will reduce the '
                             'training time if validation is slow.')
    parser.add_argument('--seed', type=int, default=233,
                        help='Random seed to be fixed.')
    parser.add_argument('--num-samples', type=int, default=-1,
                        help='Training images. Use -1 to automatically get the number.')
    parser.add_argument('--syncbn', action='store_true',
                        help='Use synchronize BN across devices.')
    parser.add_argument('--no-random-shape', action='store_true',
                        help='Use fixed size(data-shape) throughout the training, which will be faster '
                        'and require less memory. However, final model will be slightly worse.')
    parser.add_argument('--no-wd', action='store_true',
                        help='whether to remove weight decay on bias, and beta/gamma for batchnorm layers.')
    parser.add_argument('--mixup', action='store_true',
                        help='whether to enable mixup.')
    parser.add_argument('--no-mixup-epochs', type=int, default=20,
                        help='Disable mixup training if enabled in the last N epochs.')
    parser.add_argument('--label-smooth', action='store_true', help='Use label smoothing.')
    args = parser.parse_args()
    return args

def get_dataset(dataset, args):
    if dataset.lower() == 'voc':
        train_dataset = gdata.VOCDetection(
            splits=[(2007, 'trainval'), (2012, 'trainval')])
        val_dataset = gdata.VOCDetection(
            splits=[(2007, 'test')])
        val_metric = VOC07MApMetric(iou_thresh=0.5, class_names=val_dataset.classes)
    elif dataset.lower() == 'coco':
        train_dataset = gdata.COCODetection(splits='instances_train2017', use_crowd=False)
        val_dataset = gdata.COCODetection(splits='instances_val2017', skip_empty=False)
        val_metric = COCODetectionMetric(
            val_dataset, args.save_prefix + '_eval', cleanup=True,
            data_shape=(args.data_shape, args.data_shape))
    else:
        raise NotImplementedError('Dataset: {} not implemented.'.format(dataset))
    if args.num_samples < 0:
        args.num_samples = len(train_dataset)
    if args.mixup:
        from gluoncv.data import MixupDetection
        train_dataset = MixupDetection(train_dataset)
    return train_dataset, val_dataset, val_metric

def get_dataloader(net, train_dataset, val_dataset, data_shape, batch_size, num_workers, args):
    """Get dataloader."""
    width, height = data_shape, data_shape
    batchify_fn = Tuple(*([Stack() for _ in range(6)] + [Pad(axis=0, pad_val=-1) for _ in range(1)]))  # stack image, all targets generated
    if args.no_random_shape:
        train_loader = gluon.data.DataLoader(
            train_dataset.transform(YOLO3DefaultTrainTransform(width, height, net, mixup=args.mixup)),
            batch_size, True, batchify_fn=batchify_fn, last_batch='rollover', num_workers=num_workers)
    else:
        transform_fns = [YOLO3DefaultTrainTransform(x * 32, x * 32, net, mixup=args.mixup) for x in range(10, 20)]
        train_loader = RandomTransformDataLoader(
            transform_fns, train_dataset, batch_size=batch_size, interval=10, last_batch='rollover',
            shuffle=True, batchify_fn=batchify_fn, num_workers=num_workers)
    val_batchify_fn = Tuple(Stack(), Pad(pad_val=-1))
    val_loader = gluon.data.DataLoader(
        val_dataset.transform(YOLO3DefaultValTransform(width, height)),
        batch_size, False, batchify_fn=val_batchify_fn, last_batch='keep', num_workers=num_workers)
    return train_loader, val_loader

def save_params(net, best_map, current_map, epoch, save_interval, prefix):
    current_map = float(current_map)
    if current_map > best_map[0]:
        best_map[0] = current_map
        net.save_parameters('{:s}_best.params'.format(prefix, epoch, current_map))
        with open(prefix+'_best_map.log', 'a') as f:
            f.write('{:04d}:\t{:.4f}\n'.format(epoch, current_map))
    if save_interval and epoch % save_interval == 0:
        net.save_parameters('{:s}_{:04d}_{:.4f}.params'.format(prefix, epoch, current_map))

def validate(net, val_data, ctx, eval_metric):
    """Test on validation dataset."""
    eval_metric.reset()
    # set nms threshold and topk constraint
    net.set_nms(nms_thresh=0.45, nms_topk=400)
    mx.nd.waitall()
    net.hybridize()
    for batch in val_data:
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        det_bboxes = []
        det_ids = []
        det_scores = []
        gt_bboxes = []
        gt_ids = []
        gt_difficults = []
        for x, y in zip(data, label):
            # get prediction results
            ids, scores, bboxes = net(x)
            det_ids.append(ids)
            det_scores.append(scores)
            # clip to image size
            det_bboxes.append(bboxes.clip(0, batch[0].shape[2]))
            # split ground truths
            gt_ids.append(y.slice_axis(axis=-1, begin=4, end=5))
            gt_bboxes.append(y.slice_axis(axis=-1, begin=0, end=4))
            gt_difficults.append(y.slice_axis(axis=-1, begin=5, end=6) if y.shape[-1] > 5 else None)

        # update metric
        eval_metric.update(det_bboxes, det_ids, det_scores, gt_bboxes, gt_ids, gt_difficults)
    return eval_metric.get()

def train(net, train_data, val_data, eval_metric, ctx, args):
    """Training pipeline"""
    net.collect_params().reset_ctx(ctx)
    if args.no_wd:
        for k, v in net.collect_params('.*beta|.*gamma|.*bias').items():
            v.wd_mult = 0.0

    if args.label_smooth:
        net._target_generator._label_smooth = True

    if args.lr_decay_period > 0:
        lr_decay_epoch = list(range(args.lr_decay_period, args.epochs, args.lr_decay_period))
    else:
        lr_decay_epoch = [int(i) for i in args.lr_decay_epoch.split(',')]
    lr_decay_epoch = [e - args.warmup_epochs for e in lr_decay_epoch]
    num_batches = args.num_samples // args.batch_size
    lr_scheduler = LRSequential([
        LRScheduler('linear', base_lr=0, target_lr=args.lr,
                    nepochs=args.warmup_epochs, iters_per_epoch=num_batches),
        LRScheduler(args.lr_mode, base_lr=args.lr,
                    nepochs=args.epochs - args.warmup_epochs,
                    iters_per_epoch=num_batches,
                    step_epoch=lr_decay_epoch,
                    step_factor=args.lr_decay, power=2),
    ])

    trainer = gluon.Trainer(
        net.collect_params(), 'sgd',
        {'wd': args.wd, 'momentum': args.momentum, 'lr_scheduler': lr_scheduler},
        kvstore='local')

    # targets
    sigmoid_ce = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
    l1_loss = gluon.loss.L1Loss()

    # metrics
    obj_metrics = mx.metric.Loss('ObjLoss')
    center_metrics = mx.metric.Loss('BoxCenterLoss')
    scale_metrics = mx.metric.Loss('BoxScaleLoss')
    cls_metrics = mx.metric.Loss('ClassLoss')

    # set up logger
    logging.basicConfig()
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    log_file_path = args.save_prefix + '_train.log'
    log_dir = os.path.dirname(log_file_path)
    if log_dir and not os.path.exists(log_dir):
        os.makedirs(log_dir)
    fh = logging.FileHandler(log_file_path)
    logger.addHandler(fh)
    logger.info(args)
    logger.info('Start training from [Epoch {}]'.format(args.start_epoch))
    best_map = [0]
    for epoch in range(args.start_epoch, args.epochs):
        if args.mixup:
            # TODO(zhreshold): more elegant way to control mixup during runtime
            try:
                train_data._dataset.set_mixup(np.random.beta, 1.5, 1.5)
            except AttributeError:
                train_data._dataset._data.set_mixup(np.random.beta, 1.5, 1.5)
            if epoch >= args.epochs - args.no_mixup_epochs:
                try:
                    train_data._dataset.set_mixup(None)
                except AttributeError:
                    train_data._dataset._data.set_mixup(None)

        tic = time.time()
        btic = time.time()
        mx.nd.waitall()
        net.hybridize()
        for i, batch in enumerate(train_data):
            batch_size = batch[0].shape[0]
            data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            # objectness, center_targets, scale_targets, weights, class_targets
            fixed_targets = [gluon.utils.split_and_load(batch[it], ctx_list=ctx, batch_axis=0) for it in range(1, 6)]
            gt_boxes = gluon.utils.split_and_load(batch[6], ctx_list=ctx, batch_axis=0)
            sum_losses = []
            obj_losses = []
            center_losses = []
            scale_losses = []
            cls_losses = []
            with autograd.record():
                for ix, x in enumerate(data):
                    obj_loss, center_loss, scale_loss, cls_loss = net(x, gt_boxes[ix], *[ft[ix] for ft in fixed_targets])
                    sum_losses.append(obj_loss + center_loss + scale_loss + cls_loss)
                    obj_losses.append(obj_loss)
                    center_losses.append(center_loss)
                    scale_losses.append(scale_loss)
                    cls_losses.append(cls_loss)
                autograd.backward(sum_losses)
            trainer.step(batch_size)
            obj_metrics.update(0, obj_losses)
            center_metrics.update(0, center_losses)
            scale_metrics.update(0, scale_losses)
            cls_metrics.update(0, cls_losses)
            if args.log_interval and not (i + 1) % args.log_interval:
                name1, loss1 = obj_metrics.get()
                name2, loss2 = center_metrics.get()
                name3, loss3 = scale_metrics.get()
                name4, loss4 = cls_metrics.get()
                logger.info('[Epoch {}][Batch {}], LR: {:.2E}, Speed: {:.3f} samples/sec, {}={:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}'.format(
                    epoch, i, trainer.learning_rate, batch_size/(time.time()-btic), name1, loss1, name2, loss2, name3, loss3, name4, loss4))
            btic = time.time()

        name1, loss1 = obj_metrics.get()
        name2, loss2 = center_metrics.get()
        name3, loss3 = scale_metrics.get()
        name4, loss4 = cls_metrics.get()
        logger.info('[Epoch {}] Training cost: {:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}'.format(
            epoch, (time.time()-tic), name1, loss1, name2, loss2, name3, loss3, name4, loss4))
        if not (epoch + 1) % args.val_interval:
            # consider reduce the frequency of validation to save time
            map_name, mean_ap = validate(net, val_data, ctx, eval_metric)
            val_msg = '\n'.join(['{}={}'.format(k, v) for k, v in zip(map_name, mean_ap)])
            logger.info('[Epoch {}] Validation: \n{}'.format(epoch, val_msg))
            current_map = float(mean_ap[-1])
        else:
            current_map = 0.
        save_params(net, best_map, current_map, epoch, args.save_interval, args.save_prefix)

#if __name__ == '__main__':
#    args = parse_args()
#    # fix seed for mxnet, numpy and python builtin random generator.
#    gutils.random.seed(args.seed)

#    # training contexts
#    ctx = [mx.gpu(int(i)) for i in args.gpus.split(',') if i.strip()]
#    ctx = ctx if ctx else [mx.cpu()]

#    # network
#    net_name = '_'.join(('yolo3', args.network, args.dataset))
#    args.save_prefix += net_name
#    # use sync bn if specified
#    if args.syncbn and len(ctx) > 1:
#        net = get_model(net_name, pretrained_base=True, norm_layer=gluon.contrib.nn.SyncBatchNorm,
#                        norm_kwargs={'num_devices': len(ctx)})
#        async_net = get_model(net_name, pretrained_base=False)  # used by cpu worker
#    else:
#        net = get_model(net_name, pretrained_base=True)
#        async_net = net
#    if args.resume.strip():
#        net.load_parameters(args.resume.strip())
#        async_net.load_parameters(args.resume.strip())
#    else:
#        with warnings.catch_warnings(record=True) as w:
#            warnings.simplefilter("always")
#            net.initialize()
#            async_net.initialize()

#    # training data
#    train_dataset, val_dataset, eval_metric = get_dataset(args.dataset, args)
#    train_data, val_data = get_dataloader(
#        async_net, train_dataset, val_dataset, args.data_shape, args.batch_size, args.num_workers, args)

#    # training
#    train(net, train_data, val_data, eval_metric, ctx, args)

In [ ]:
args = parse_args()
# fix seed for mxnet, numpy and python builtin random generator.
gutils.random.seed(1)

# training contexts
#ctx = [mx.gpu(int(i)) for i in args.gpus.split(',') if i.strip()]
ctx = ctx if ctx else [mx.cpu()]
ctx = [mx.cpu()]

# network
net_name = '_'.join(('yolo3', args.network, args.dataset))
args.save_prefix += net_name
# use sync bn if specified
if args.syncbn and len(ctx) > 1:
    net = get_model(net_name, pretrained_base=True, norm_layer=gluon.contrib.nn.SyncBatchNorm,
                    norm_kwargs={'num_devices': len(ctx)})
    async_net = get_model(net_name, pretrained_base=False)  # used by cpu worker
else:
    net = get_model(net_name, pretrained_base=True)
    async_net = net
if args.resume.strip():
    net.load_parameters(args.resume.strip())
    async_net.load_parameters(args.resume.strip())
else:
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")
        net.initialize()
        async_net.initialize()

# training data
train_dataset, val_dataset, eval_metric = get_dataset(args.dataset, args)
train_data, val_data = get_dataloader(
    async_net, train_dataset, val_dataset, args.data_shape, args.batch_size, args.num_workers, args)

# training
train(net, train_data, val_data, eval_metric, ctx, args)

In [1]:
from __future__ import division
from __future__ import print_function

import argparse
import logging
logging.basicConfig(level=logging.INFO)
import time
import numpy as np
import mxnet as mx
from tqdm import tqdm
from mxnet import nd
from mxnet import gluon
import gluoncv as gcv
from gluoncv import data as gdata
from gluoncv.data.batchify import Tuple, Stack, Pad
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultValTransform
from gluoncv.utils.metrics.voc_detection import VOC07MApMetric
from gluoncv.utils.metrics.coco_detection import COCODetectionMetric


def parse_args():
    parser = argparse.ArgumentParser(description='Eval YOLO networks.')
    parser.add_argument('--network', type=str, default='darknet53',
                        help="Base network name")
    parser.add_argument('--algorithm', type=str, default='yolo3',
                        help='YOLO version, default is yolo3')
    parser.add_argument('--data-shape', type=int, default=416,
                        help="Input data shape")
    parser.add_argument('--batch-size', type=int, default=64,
                        help='Training mini-batch size')
    parser.add_argument('--dataset', type=str, default='voc',
                        help='Training dataset.')
    parser.add_argument('--num-workers', '-j', dest='num_workers', type=int,
                        default=4, help='Number of data workers')
    parser.add_argument('--gpus', type=str, default='0',
                        help='Training with GPUs, you can specify 1,3 for example.')
    parser.add_argument('--pretrained', type=str, default='True',
                        help='Load weights from previously saved parameters.')
    parser.add_argument('--save-prefix', type=str, default='',
                        help='Saving parameter prefix')
    args = parser.parse_args()
    return args

def get_dataset(dataset, data_shape):
    if dataset.lower() == 'voc':
        val_dataset = gdata.VOCDetection(splits=[(2007, 'test')])
        val_metric = VOC07MApMetric(iou_thresh=0.5, class_names=val_dataset.classes)
    elif dataset.lower() == 'coco':
        val_dataset = gdata.COCODetection(splits='instances_val2017', skip_empty=False)
        val_metric = COCODetectionMetric(
            val_dataset, args.save_prefix + '_eval', cleanup=True,
            data_shape=(data_shape, data_shape))
    else:
        raise NotImplementedError('Dataset: {} not implemented.'.format(dataset))
    return val_dataset, val_metric

def get_dataloader(val_dataset, data_shape, batch_size, num_workers):
    """Get dataloader."""
    width, height = data_shape, data_shape
    batchify_fn = Tuple(Stack(), Pad(pad_val=-1))
    val_loader = gluon.data.DataLoader(
        val_dataset.transform(YOLO3DefaultValTransform(width, height)),
        batch_size, False, last_batch='keep', num_workers=num_workers, batchify_fn=batchify_fn,)
    return val_loader

def validate(net, val_data, ctx, classes, size, metric):
    """Test on validation dataset."""
    net.collect_params().reset_ctx(ctx)
    metric.reset()
    net.set_nms(nms_thresh=0.45, nms_topk=400)
    net.hybridize()
    with tqdm(total=size) as pbar:
        for ib, batch in enumerate(val_data):
            data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
            label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
            det_bboxes = []
            det_ids = []
            det_scores = []
            gt_bboxes = []
            gt_ids = []
            gt_difficults = []
            for x, y in zip(data, label):
                ids, scores, bboxes = net(x)
                det_ids.append(ids)
                det_scores.append(scores)
                # clip to image size
                det_bboxes.append(bboxes.clip(0, batch[0].shape[2]))
                # split ground truths
                gt_ids.append(y.slice_axis(axis=-1, begin=4, end=5))
                gt_bboxes.append(y.slice_axis(axis=-1, begin=0, end=4))
                gt_difficults.append(y.slice_axis(axis=-1, begin=5, end=6) if y.shape[-1] > 5 else None)

            metric.update(det_bboxes, det_ids, det_scores, gt_bboxes, gt_ids, gt_difficults)
            pbar.update(batch[0].shape[0])
    return metric.get()

#if __name__ == '__main__':
#    args = parse_args()
#
#    # training contexts
#    ctx = [mx.gpu(int(i)) for i in args.gpus.split(',') if i.strip()]
#    ctx = ctx if ctx else [mx.cpu()]

#    # network
#    net_name = '_'.join((args.algorithm, args.network, args.dataset))
#    args.save_prefix += net_name
#    if args.pretrained.lower() in ['true', '1', 'yes', 't']:
#        net = gcv.model_zoo.get_model(net_name, pretrained=True)
#    else:
#        net = gcv.model_zoo.get_model(net_name, pretrained=False)
#        net.load_parameters(args.pretrained.strip())

#    # training data
#    val_dataset, val_metric = get_dataset(args.dataset, args.data_shape)
#    val_data = get_dataloader(
#        val_dataset, args.data_shape, args.batch_size, args.num_workers)
#    classes = val_dataset.classes  # class names

#    # training
#    names, values = validate(net, val_data, ctx, classes, len(val_dataset), val_metric)
#    for k, v in zip(names, values):
#        print(k, v)

In [2]:
start = time.time()

In [3]:
#args = parse_args()

## training contexts
#ctx = [mx.gpu(int(i)) for i in args.gpus.split(',') if i.strip()]
#ctx = ctx if ctx else [mx.cpu()]
ctx = [mx.cpu()]

# network
#net_name = '_'.join((args.algorithm, args.network, args.dataset))
#args.save_prefix += net_name
#if args.pretrained.lower() in ['true', '1', 'yes', 't']:
#    net = gcv.model_zoo.get_model(net_name, pretrained=True)
#else:
#    net = gcv.model_zoo.get_model(net_name, pretrained=False)
#    net.load_parameters(args.pretrained.strip())
    


# training data
#val_dataset, val_metric = get_dataset(args.dataset, args.data_shape)
#val_data = get_dataloader(val_dataset, args.data_shape, args.batch_size, args.num_workers)
#classes = val_dataset.classes  # class names

net_name = 'ssd_512_resnet50_v1_coco'
net = gcv.model_zoo.get_model(net_name, pretrained=True)
data_shape = 512
batch_size = 1
num_workers = 1
val_dataset = gdata.COCODetection(root='/mnt/muthderd/MIDS/W210/data/', splits='/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_val2017', skip_empty=False)
val_data = get_dataloader(val_dataset, data_shape, batch_size, num_workers)
#val_dataset = gdata.COCODetection(splits='instances_val2017', skip_empty=False)
val_metric = COCODetectionMetric(val_dataset, 'test_eval', cleanup=True, data_shape=(data_shape, data_shape))
classes = val_dataset.classes  # class names

# training
names, values = validate(net, val_data, ctx, classes, len(val_dataset), val_metric)
for k, v in zip(names, values):
    print(k, v)

loading annotations into memory...
Done (t=0.29s)
creating index...
index created!


100%|██████████| 5000/5000 [27:50<00:00,  3.03it/s]


Loading and preparing results...
DONE (t=0.97s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=22.19s).
Accumulating evaluation results...
DONE (t=2.87s).
~~~~ Summary metrics ~~~~
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.305
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.497
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.357
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.481
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.276
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.405
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.422
 Average Recall     (AR) @[ IoU=0.50:0.95

In [4]:
end = time.time()

In [5]:
end - start

1702.4682216644287